In [ ]:
!pip install Sastrawi==1.0.1
!pip install indonlp==0.3.4
!pip install swifter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=3a75528194090cbf7f041ce9db7834a456f1426dc2e9e1f3bf173d86e8bf9fe4
  Stored in directory: /root/.cache/pip/wheels/d9/31/ff/ff51141a088571a9f672449e5aad5ea8bb35ca5d95ba135f30
Successfully built swifter


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Skripsi

/content/drive/MyDrive/Skripsi


In [ ]:
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import swifter
from tqdm import tqdm
import re
import os
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from indoNLP.preprocessing import replace_slang
from functools import lru_cache
from multiprocessing import Pool, cpu_count
import gc
import utils.sentimen_processing as sp

In [ ]:
print(tf.__version__)

2.19.0


In [ ]:
os.makedirs(f"data/backup", exist_ok=True)

## Persiapan

In [ ]:
sentimen_model = tf.keras.models.load_model("model/model_sentimen.keras")

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 13 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
#inialization Steamer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
with open("data/stopWords.txt", "r", encoding="utf-8") as f:
    stopWords = set(f.read().splitlines())
print(stopWords)

{'unvr', 'username', 'url', 'indomie', 'siap', 'kan', 'indonesia', 'akan', '[url]', 'gojek', 'bila', 'ini', 'bri', 'adalah', 'dia', 'ibu', 'bnni', 'kata', 'per', 'grab', 'beri', 'tentang', 'ia', 'pak', 'itu', 'indf', 'kami', 'toh', 'dengan', 'bapak', 'bumn', 'ojk', 'yang', 'yuk', 'pun', 'agar', 'saya', 'tlkm', 'semua', 'untuk', 'bahwa', 'bagai', 'buat', 'se', 'asal', 'begini', 'saja', 'nya', 'berupa', 'yaitu', 'kapan', 'bmri', 'telkomsel', 'dalam', 'sebab', 'mereka', '[username]', 'jika', 'serta', 'hashtag', 'ri', 'kwong', 'supaya', 'wahai', 'goto', 'maka', 'atau', 'kamu', 'indofood', 'antar', 'mirip', 'telkom', 'bagi', 'emiten', 'asi', 'di', 'tokopedia', 'para', 'kepada', 'bbri', 'aku', 'bung', '[hashtag]', 'tuck', 'oleh', 'ikut', 'berapa', 'diri', 'yakni', 'dan', 'walau', 'sebagai', 'anda', 'memang', 'kini', 'sehingga', 'cara', 'kok', 'saat', 'kita', 'pihak'}


In [ ]:
with open("data/vectorizer_vocab.pkl", "rb") as f:
    vocab = pickle.load(f)

# buat ulang vectorizer
vectorizer = tf.keras.layers.TextVectorization(
    output_sequence_length=100,
    vocabulary=vocab
)

vocabs = vectorizer.get_vocabulary()
print("Vocabulary:", vocabs)
print("Vocabulary size:", len(vocabs))

Vocabulary: ['', '[UNK]', np.str_('num'), np.str_('saham'), np.str_('rp'), np.str_('tidak'), np.str_('makan'), np.str_('ke'), np.str_('jadi'), np.str_('ada'), np.str_('dari'), np.str_('harga'), np.str_('ihsg'), np.str_('bank'), np.str_('tempat'), np.str_('enak'), np.str_('sama'), np.str_('sudah'), np.str_('bisa'), np.str_('juga'), np.str_('tapi'), np.str_('banyak'), np.str_('buka'), np.str_('rasa'), np.str_('lagi'), np.str_('orang'), np.str_('naik'), np.str_('karena'), np.str_('tahun'), np.str_('mau'), np.str_('kalau'), np.str_('hari'), np.str_('sangat'), np.str_('banget'), np.str_('pada'), np.str_('baru'), np.str_('layan'), np.str_('lebih'), np.str_('t'), np.str_('sini'), np.str_('masih'), np.str_('dapat'), np.str_('ya'), np.str_('beli'), np.str_('menu'), np.str_('pilih'), np.str_('asing'), np.str_('kuat'), np.str_('bandung'), np.str_('baik'), np.str_('uang'), np.str_('sih'), np.str_('bos'), np.str_('besar'), np.str_('turun'), np.str_('cukup'), np.str_('triliun'), np.str_('kerja'), np

## TLKM

In [ ]:
ticker = 'TLKM'

### Load Data

In [ ]:
sentimenX = pd.read_csv(f"Scrapping_Data/{ticker}/data/Full_X_comments_{ticker}.csv")
sentimenX

,tanggal,sentimen
0,2022-07-29,Pengunjung dapat merasakan sensasi teknologi m...
1,2022-07-29,Telkom Bikin metaNesia Dunia Metaverse Berisi ...
2,2022-07-29,Telkom Indonesia mulai kembangkan metaverse de...
3,2022-07-29,#DigitalBisa #UntukIndonesiaLebihBaik #Digital...
4,2022-07-29,Badan Usaha Milik Negara Telkom Indonesia (Per...
...,...,...
145140,2024-12-01,Telkomsel ngeselin amat ya lemot bgt gini hmmm
145141,2024-12-01,@HarunAr52826799 @HarunAr52826799 Halo Kak Har...
145142,2024-12-01,@tash06042823 @tash06042823 Mohon maaf ya biki...
145143,2024-12-01,@IndiHomeCare @Telkomsel Halo tumbenan lama re...


In [ ]:
sentimenYT = pd.read_csv(f"Scrapping_Data/{ticker}/data/youtube_comments_{ticker}.csv")
sentimenYT

,tanggal,sentimen
0,2023-05-05,Makasih bang 100% work
1,2023-05-05,mantab kaka
2,2023-07-15,Ya work lah kartu lu kan kartu sakti
3,2023-07-21,@@serbaserbimobildanpenjuala4958 iyaa sebenarn...
4,2023-08-14,​@@daftar_tutorial😮😮
...,...,...
224402,2024-06-25,"Itu yg kak Dita main mainan buaya, yg Abang Su..."
224403,2024-06-25,https://youtu.be/tDkTNeghQi4?si=HGgCfz8y64xgRi6l
224404,2024-06-25,Yang ada ka maudy
224405,2024-06-25,Kapan tayang lagi critnya kangen❤❤❤


In [ ]:
sentimenDF = pd.concat([sentimenX, sentimenYT], ignore_index=True)
sentimenDF

,tanggal,sentimen
0,2022-07-29,Pengunjung dapat merasakan sensasi teknologi m...
1,2022-07-29,Telkom Bikin metaNesia Dunia Metaverse Berisi ...
2,2022-07-29,Telkom Indonesia mulai kembangkan metaverse de...
3,2022-07-29,#DigitalBisa #UntukIndonesiaLebihBaik #Digital...
4,2022-07-29,Badan Usaha Milik Negara Telkom Indonesia (Per...
...,...,...
369547,2024-06-25,"Itu yg kak Dita main mainan buaya, yg Abang Su..."
369548,2024-06-25,https://youtu.be/tDkTNeghQi4?si=HGgCfz8y64xgRi6l
369549,2024-06-25,Yang ada ka maudy
369550,2024-06-25,Kapan tayang lagi critnya kangen❤❤❤


### Cek data null dan duplikat

In [ ]:
sentimenDF.duplicated().sum()

np.int64(22839)

In [ ]:
sentimenDF.isna().sum()

,0
tanggal,5
sentimen,7


In [ ]:
sentimenDF.dropna(inplace=True)
sentimenDF.drop_duplicates(inplace=True)

In [ ]:
print(sentimenDF.duplicated().sum())
sentimenDF.isna().sum()

0


,0
tanggal,0
sentimen,0


### preprocessing data

In [ ]:
sentimenDF['clean_text'] = sp.preprocess_df(sentimenDF['sentimen'])

In [ ]:
sentimenDF

,tanggal,sentimen,clean_text
0,2022-07-29,Pengunjung dapat merasakan sensasi teknologi m...,pengunjung dapat merasakan sensasi teknologi m...
1,2022-07-29,Telkom Bikin metaNesia Dunia Metaverse Berisi ...,telkom bikin metanesia dunia metaverse berisi ...
2,2022-07-29,Telkom Indonesia mulai kembangkan metaverse de...,telkom indonesia mulai kembangkan metaverse de...
3,2022-07-29,#DigitalBisa #UntukIndonesiaLebihBaik #Digital...,melalui leap-telkom telkom indonesia akan c...
4,2022-07-29,Badan Usaha Milik Negara Telkom Indonesia (Per...,badan usaha milik negara telkom indonesia per...
...,...,...,...
369547,2024-06-25,"Itu yg kak Dita main mainan buaya, yg Abang Su...",itu yg kak dita main mainan buaya yg abang su...
369548,2024-06-25,https://youtu.be/tDkTNeghQi4?si=HGgCfz8y64xgRi6l,
369549,2024-06-25,Yang ada ka maudy,yang ada ka maudy
369550,2024-06-25,Kapan tayang lagi critnya kangen❤❤❤,kapan tayang lagi critnya kangen❤❤❤


In [ ]:
sentimenDF = sp.parallel_processing(sentimenDF)

🔄 Main Progress: 100%|██████████| 18/18 [5:17:19<00:00, 1057.72s/it]


In [ ]:
sentimenDF

,tanggal,sentimen,clean_text,text_final
0,2022-07-29,Pengunjung dapat merasakan sensasi teknologi m...,pengunjung dapat merasakan sensasi teknologi m...,unjung dapat rasa sensasi teknologi metaverse
1,2022-07-29,Telkom Bikin metaNesia Dunia Metaverse Berisi ...,telkom bikin metanesia dunia metaverse berisi ...,bikin metanesia dunia metaverse isi muat lokal
2,2022-07-29,Telkom Indonesia mulai kembangkan metaverse de...,telkom indonesia mulai kembangkan metaverse de...,mulai kembang metaverse nama metaverse sebut m...
3,2022-07-29,#DigitalBisa #UntukIndonesiaLebihBaik #Digital...,melalui leap-telkom telkom indonesia akan c...,lalu leap cipta dunia metaverse bisa manfaat m...
4,2022-07-29,Badan Usaha Milik Negara Telkom Indonesia (Per...,badan usaha milik negara telkom indonesia per...,badan usaha milik negara persero lalu leap waj...
...,...,...,...,...
369547,2024-06-25,"Itu yg kak Dita main mainan buaya, yg Abang Su...",itu yg kak dita main mainan buaya yg abang su...,kak dita main main buaya abang surya hampir pe...
369548,2024-06-25,https://youtu.be/tDkTNeghQi4?si=HGgCfz8y64xgRi6l,,
369549,2024-06-25,Yang ada ka maudy,yang ada ka maudy,ada kak maudy
369550,2024-06-25,Kapan tayang lagi critnya kangen❤❤❤,kapan tayang lagi critnya kangen❤❤❤,tayang lagi critnya kangen


### Simpan Data

In [ ]:
sentimenDF.to_csv(f"data/{ticker}/sentimen_preprocessing_{ticker}.csv", index=False)

### prediksi nilai sentimen

In [ ]:
sentimenDF = pd.read_csv(f"data/{ticker}/sentimen_preprocessing_{ticker}.csv")
sentimenDF

,tanggal,sentimen,clean_text,text_final
0,2022-07-29,Pengunjung dapat merasakan sensasi teknologi m...,pengunjung dapat merasakan sensasi teknologi m...,unjung dapat rasa sensasi teknologi metaverse
1,2022-07-29,Telkom Bikin metaNesia Dunia Metaverse Berisi ...,telkom bikin metanesia dunia metaverse berisi ...,bikin metanesia dunia metaverse isi muat lokal
2,2022-07-29,Telkom Indonesia mulai kembangkan metaverse de...,telkom indonesia mulai kembangkan metaverse de...,mulai kembang metaverse nama metaverse sebut m...
3,2022-07-29,#DigitalBisa #UntukIndonesiaLebihBaik #Digital...,melalui leap-telkom telkom indonesia akan c...,lalu leap cipta dunia metaverse bisa manfaat m...
4,2022-07-29,Badan Usaha Milik Negara Telkom Indonesia (Per...,badan usaha milik negara telkom indonesia per...,badan usaha milik negara persero lalu leap waj...
...,...,...,...,...
346700,2024-06-25,"Itu yg kak Dita main mainan buaya, yg Abang Su...",itu yg kak dita main mainan buaya yg abang su...,kak dita main main buaya abang surya hampir pe...
346701,2024-06-25,https://youtu.be/tDkTNeghQi4?si=HGgCfz8y64xgRi6l,NaN,NaN
346702,2024-06-25,Yang ada ka maudy,yang ada ka maudy,ada kak maudy
346703,2024-06-25,Kapan tayang lagi critnya kangen❤❤❤,kapan tayang lagi critnya kangen❤❤❤,tayang lagi critnya kangen


In [ ]:
sentimenDF = sentimenDF[["tanggal", "text_final"]].copy()
sentimenDF

,tanggal,text_final
0,2022-07-29,unjung dapat rasa sensasi teknologi metaverse
1,2022-07-29,bikin metanesia dunia metaverse isi muat lokal
2,2022-07-29,mulai kembang metaverse nama metaverse sebut m...
3,2022-07-29,lalu leap cipta dunia metaverse bisa manfaat m...
4,2022-07-29,badan usaha milik negara persero lalu leap waj...
...,...,...
346700,2024-06-25,kak dita main main buaya abang surya hampir pe...
346701,2024-06-25,NaN
346702,2024-06-25,ada kak maudy
346703,2024-06-25,tayang lagi critnya kangen


In [ ]:
sentimenDF.isna().sum()

,0
tanggal,0
text_final,4441


In [ ]:
sentimenDF.dropna(inplace=True)
sentimenDF = sentimenDF[sentimenDF['text_final'] != ''].reset_index(drop=True)

In [ ]:
sentimenDF

,tanggal,text_final
0,2022-07-29,unjung dapat rasa sensasi teknologi metaverse
1,2022-07-29,bikin metanesia dunia metaverse isi muat lokal
2,2022-07-29,mulai kembang metaverse nama metaverse sebut m...
3,2022-07-29,lalu leap cipta dunia metaverse bisa manfaat m...
4,2022-07-29,badan usaha milik negara persero lalu leap waj...
...,...,...
342259,2024-06-25,nih crt upload lagi
342260,2024-06-25,kak dita main main buaya abang surya hampir pe...
342261,2024-06-25,ada kak maudy
342262,2024-06-25,tayang lagi critnya kangen


In [ ]:
sentimenDF["prediksi_sentimen"] = sp.predict_sentimen(sentimenDF, sentimen_model, vectorizer, ticker)

100%|██████████| 69/69 [01:37<00:00,  1.41s/it]


In [ ]:
sentimenDF

,tanggal,text_final,prediksi_sentimen
0,2022-07-29,unjung dapat rasa sensasi teknologi metaverse,2
1,2022-07-29,bikin metanesia dunia metaverse isi muat lokal,1
2,2022-07-29,mulai kembang metaverse nama metaverse sebut m...,1
3,2022-07-29,lalu leap cipta dunia metaverse bisa manfaat m...,1
4,2022-07-29,badan usaha milik negara persero lalu leap waj...,1
...,...,...,...
342259,2024-06-25,nih crt upload lagi,2
342260,2024-06-25,kak dita main main buaya abang surya hampir pe...,1
342261,2024-06-25,ada kak maudy,1
342262,2024-06-25,tayang lagi critnya kangen,2


In [ ]:
sentimenDF.to_csv(f"data/{ticker}/sentimen_predict_{ticker}.csv", index=False)

### Integrasi Data

In [ ]:
print(ticker)
sentimenDF = pd.read_csv(f"data/{ticker}/sentimen_predict_{ticker}.csv")
sentimenDF

TLKM


,tanggal,text_final,prediksi_sentimen
0,2022-07-29,unjung dapat rasa sensasi teknologi metaverse,2
1,2022-07-29,bikin metanesia dunia metaverse isi muat lokal,1
2,2022-07-29,mulai kembang metaverse nama metaverse sebut m...,1
3,2022-07-29,lalu leap cipta dunia metaverse bisa manfaat m...,1
4,2022-07-29,badan usaha milik negara persero lalu leap waj...,1
...,...,...,...
342259,2024-06-25,nih crt upload lagi,2
342260,2024-06-25,kak dita main main buaya abang surya hampir pe...,1
342261,2024-06-25,ada kak maudy,1
342262,2024-06-25,tayang lagi critnya kangen,2


In [ ]:
sentimenMAP = sentimenDF.copy()
mapping = {2: 1, 1 : 0, 0 : -1}

sentimenMAP['prediksi_sentimen'] = sentimenMAP['prediksi_sentimen'].map(mapping)
sentimenMAP

,tanggal,text_final,prediksi_sentimen
0,2022-07-29,unjung dapat rasa sensasi teknologi metaverse,1
1,2022-07-29,bikin metanesia dunia metaverse isi muat lokal,0
2,2022-07-29,mulai kembang metaverse nama metaverse sebut m...,0
3,2022-07-29,lalu leap cipta dunia metaverse bisa manfaat m...,0
4,2022-07-29,badan usaha milik negara persero lalu leap waj...,0
...,...,...,...
342259,2024-06-25,nih crt upload lagi,1
342260,2024-06-25,kak dita main main buaya abang surya hampir pe...,0
342261,2024-06-25,ada kak maudy,0
342262,2024-06-25,tayang lagi critnya kangen,1


In [ ]:
sentimenDF['prediksi_sentimen'].value_counts()

,count
prediksi_sentimen,
1,165794
0,106543
2,69927


In [ ]:
sentimenMAP['prediksi_sentimen'].value_counts()

,count
prediksi_sentimen,
0,165794
-1,106543
1,69927


In [ ]:
sentimenMAP["tanggal"] = pd.to_datetime(sentimenMAP["tanggal"])

df_mean = (
    sentimenMAP
    .groupby("tanggal", as_index=False)["prediksi_sentimen"]
    .mean()
)

df_mean = df_mean.sort_values("tanggal")

df_mean.rename(columns={"prediksi_sentimen": "nilai_sentimen"}, inplace=True)

df_mean

,tanggal,nilai_sentimen
0,2007-03-26,0.000000
1,2009-12-31,0.000000
2,2011-03-02,0.000000
3,2011-03-05,-1.000000
4,2011-03-08,1.000000
...,...,...
3312,2025-10-15,-0.229167
3313,2025-10-16,0.097561
3314,2025-10-17,-0.200000
3315,2025-10-18,0.000000


In [ ]:
historisDF = pd.read_csv(
    f"./data/{ticker}/{ticker}.csv",
    index_col=0,
    parse_dates=[0],
    date_format="%Y-%m-%d"
    )

historisDF = historisDF[historisDF.index.str.match(r'\d{4}-\d{2}-\d{2}', na=False)]
historisDF.index = pd.to_datetime(historisDF.index)
historisDF.index.name = 'tanggal'

historisDF

,Close,High,Low,Open,Volume
tanggal,,,,,
2022-07-11,3386.84814453125,3437.147869450031,3353.314994585396,3437.147869450031,42258000
2022-07-12,3361.6982421875,3403.6146791225065,3361.6982421875,3403.6146791225065,43399800
2022-07-13,3303.01513671875,3378.464721060041,3294.6318495697174,3361.698146761976,96218200
2022-07-14,3370.08154296875,3370.08154296875,3269.4820939249066,3303.0152436061876,68708500
2022-07-15,3479.064208984375,3529.3639324877636,3361.6981874764683,3386.848049228163,146093800
...,...,...,...,...,...
2025-07-25,2820.0,2830.0,2780.0,2810.0,41296200
2025-07-28,2860.0,2870.0,2830.0,2870.0,40193800
2025-07-29,2880.0,2880.0,2840.0,2870.0,46357400


In [ ]:
integrasiDF = pd.merge(
    historisDF,
    df_mean,
    on="tanggal",
    how="left"
)

integrasiDF

,tanggal,Close,High,Low,Open,Volume,nilai_sentimen
0,2022-07-11,3386.84814453125,3437.147869450031,3353.314994585396,3437.147869450031,42258000,-0.234899
1,2022-07-12,3361.6982421875,3403.6146791225065,3361.6982421875,3403.6146791225065,43399800,-0.116959
2,2022-07-13,3303.01513671875,3378.464721060041,3294.6318495697174,3361.698146761976,96218200,-0.230263
3,2022-07-14,3370.08154296875,3370.08154296875,3269.4820939249066,3303.0152436061876,68708500,-0.182482
4,2022-07-15,3479.064208984375,3529.3639324877636,3361.6981874764683,3386.848049228163,146093800,-0.158824
...,...,...,...,...,...,...,...
727,2025-07-25,2820.0,2830.0,2780.0,2810.0,41296200,-0.066406
728,2025-07-28,2860.0,2870.0,2830.0,2870.0,40193800,-0.036199
729,2025-07-29,2880.0,2880.0,2840.0,2870.0,46357400,-0.192308
730,2025-07-30,2880.0,2890.0,2850.0,2890.0,99062400,0.000000


In [ ]:
integrasiDF.isna().sum()

,0
tanggal,0
Close,0
High,0
Low,0
Open,0
Volume,0
nilai_sentimen,0


In [ ]:
integrasiDF["nilai_sentimen"] = integrasiDF["nilai_sentimen"].fillna(0)

In [ ]:
integrasiDF = integrasiDF.set_index("tanggal")
integrasiDF

,Close,High,Low,Open,Volume,nilai_sentimen
tanggal,,,,,,
2022-07-11,3386.84814453125,3437.147869450031,3353.314994585396,3437.147869450031,42258000,-0.234899
2022-07-12,3361.6982421875,3403.6146791225065,3361.6982421875,3403.6146791225065,43399800,-0.116959
2022-07-13,3303.01513671875,3378.464721060041,3294.6318495697174,3361.698146761976,96218200,-0.230263
2022-07-14,3370.08154296875,3370.08154296875,3269.4820939249066,3303.0152436061876,68708500,-0.182482
2022-07-15,3479.064208984375,3529.3639324877636,3361.6981874764683,3386.848049228163,146093800,-0.158824
...,...,...,...,...,...,...
2025-07-25,2820.0,2830.0,2780.0,2810.0,41296200,-0.066406
2025-07-28,2860.0,2870.0,2830.0,2870.0,40193800,-0.036199
2025-07-29,2880.0,2880.0,2840.0,2870.0,46357400,-0.192308


In [ ]:
integrasiDF.to_csv(f"data/{ticker}/data_integrasi_{ticker}.csv")

## INDF

In [ ]:
ticker = 'INDF'

### Load Data

In [ ]:
sentimenX = pd.read_csv(f"Scrapping_Data/{ticker}/data/Full_X_comments_{ticker}.csv")
sentimenX

,tanggal,sentimen
0,2022-07-01,@cahbola69 Sementara yg sudah fix brp sponsor ...
1,2022-07-01,@360drajat @worksfess Sales apa dulu cobain de...
2,2022-07-01,@indofood_cbp @Indomielovers Halo min @indofoo...
3,2022-07-01,@lotekdevotee Indofood bakal bangkrut kayaknya...
4,2022-07-01,@MachanKertas Nu Indofood gak sih kalo lo.
...,...,...
15592,2025-07-31,@andrew_bendico @randomable_ @indofood @BPOM_R...
15593,2025-07-31,@by_inkriya Ini mind blowing sekali ya. Gak ou...
15594,2025-07-31,@andrew_bendico @randomable_ @indofood @BPOM_R...
15595,2025-07-31,Butuh beli untuk pindah: 1. Koper 28 1 unit 2....


In [ ]:
sentimenYT = pd.read_csv(f"Scrapping_Data/{ticker}/data/youtube_comments_{ticker}.csv")
sentimenYT

,tanggal,sentimen
0,2019-02-24,contoh jika aku punya uang 120.000.000. 36 jut...
1,2025-07-05,"Boikot toko dan produknya salim grup, yes. No ..."
2,2025-06-24,"Yg mau baikot harus asli impor dari Israel, ba..."
3,2025-06-23,"terima kasih atas info nya,"
4,2025-06-17,i ?
...,...,...
273828,2025-10-11,Keempat
273829,2025-10-11,Kedua
273830,2025-10-11,Kedua
273831,2025-10-11,Kedua


In [ ]:
sentimenDF = pd.concat([sentimenX, sentimenYT], ignore_index=True)
sentimenDF

,tanggal,sentimen
0,2022-07-01,@cahbola69 Sementara yg sudah fix brp sponsor ...
1,2022-07-01,@360drajat @worksfess Sales apa dulu cobain de...
2,2022-07-01,@indofood_cbp @Indomielovers Halo min @indofoo...
3,2022-07-01,@lotekdevotee Indofood bakal bangkrut kayaknya...
4,2022-07-01,@MachanKertas Nu Indofood gak sih kalo lo.
...,...,...
289425,2025-10-11,Keempat
289426,2025-10-11,Kedua
289427,2025-10-11,Kedua
289428,2025-10-11,Kedua


### Cek data null dan duplikat

In [ ]:
sentimenDF.duplicated().sum()

np.int64(14760)

In [ ]:
sentimenDF.isna().sum()

,0
tanggal,0
sentimen,7


In [ ]:
sentimenDF.dropna(inplace=True)
sentimenDF.drop_duplicates(inplace=True)

In [ ]:
print(sentimenDF.duplicated().sum())
sentimenDF.isna().sum()

0


,0
tanggal,0
sentimen,0


### preprocessing data

In [ ]:
sentimenDF['clean_text'] = sp.preprocess_df(sentimenDF['sentimen'])

In [ ]:
sentimenDF

,tanggal,sentimen,clean_text
0,2022-07-01,@cahbola69 Sementara yg sudah fix brp sponsor ...,sementara yg sudah fix brp sponsor mas? kayan...
1,2022-07-01,@360drajat @worksfess Sales apa dulu cobain de...,sales apa dulu cobain deh sales di perusahaa...
2,2022-07-01,@indofood_cbp @Indomielovers Halo min @indofoo...,halo min saya ingin bertanya apakah saat in...
3,2022-07-01,@lotekdevotee Indofood bakal bangkrut kayaknya...,indofood bakal bangkrut kayaknya dan bakal be...
4,2022-07-01,@MachanKertas Nu Indofood gak sih kalo lo.,nu indofood gak sih kalo lo
...,...,...,...
289422,2025-10-11,😂,😂
289423,2025-10-11,Ke 7😊😊😊,ke <num> 😊😊😊
289424,2025-10-11,15 mnt yang lalu🎉,<num> mnt yang lalu🎉
289425,2025-10-11,Keempat,keempat


In [ ]:
sentimenDF = sp.parallel_processing(sentimenDF)

Memproses 274663 data dengan 2 pekerja.



🔄 Main Progress:   0%|          | 0/14 [00:00<?, ?it/s]

Batch 1:   0%|          | 0/20000 [00:00<?, ?it/s]

Batch 1:   0%|          | 1/20000 [06:24<2133:34:41, 384.06s/it]

Batch 1:   5%|▌         | 1001/20000 [07:26<1:44:56,  3.02it/s] 

Batch 1:   5%|▌         | 1001/20000 [07:41<1:44:56,  3.02it/s]

Batch 1:  10%|█         | 2001/20000 [12:14<1:31:43,  3.27it/s]

Batch 1:  15%|█▌        | 3001/20000 [13:22<55:53,  5.07it/s]  

Batch 1:  15%|█▌        | 3001/20000 [13:41<55:53,  5.07it/s]

Batch 1:  20%|██        | 4001/20000 [17:18<56:40,  4.71it/s]

Batch 1:  25%|██▌       | 5001/20000 [18:42<41:32,  6.02it/s]

Batch 1:  25%|██▌       | 5001/20000 [19:01<41:32,  6.02it/s]

Batch 1:  30%|███       | 6001/20000 [22:49<45:11,  5.16it/s]

Batch 1:  35%|███▌      | 7001/20000 [24:20<34:40,  6.25it/s]

Batch 1:  35%|███▌      | 7001/20000 [24:31<34:40,  6.25it/s]

Batch 1:  40%|████      | 8001/20000 [28:12<36:36,  5.46it/s]

Batch 1:  45%|████▌     | 9001/20000 [29:18<26:50,  6.83it/

In [ ]:
sentimenDF

,tanggal,sentimen,clean_text,text_final
0,2022-07-01,@cahbola69 Sementara yg sudah fix brp sponsor ...,sementara yg sudah fix brp sponsor mas? kayan...,sementara sudah fix sponsor mas kayane lagi tok
1,2022-07-01,@360drajat @worksfess Sales apa dulu cobain de...,sales apa dulu cobain deh sales di perusahaa...,sales apa dulu coba deh sales usaha fmcg kayak...
2,2022-07-01,@indofood_cbp @Indomielovers Halo min @indofoo...,halo min saya ingin bertanya apakah saat in...,halo min ingin tanya apakah pt sedang buka aju...
3,2022-07-01,@lotekdevotee Indofood bakal bangkrut kayaknya...,indofood bakal bangkrut kayaknya dan bakal be...,bakal bangkrut kayak bakal dampak negara juga ...
4,2022-07-01,@MachanKertas Nu Indofood gak sih kalo lo.,nu indofood gak sih kalo lo,nu enggak sih kalo lo
...,...,...,...,...
289422,2025-10-11,😂,😂,
289423,2025-10-11,Ke 7😊😊😊,ke <num> 😊😊😊,ke num
289424,2025-10-11,15 mnt yang lalu🎉,<num> mnt yang lalu🎉,num minta lalu
289425,2025-10-11,Keempat,keempat,empat


### Simpan Data

In [ ]:
sentimenDF.to_csv(f"data/{ticker}/sentimen_preprocessing_{ticker}.csv", index=False)

### prediksi nilai sentimen

In [ ]:
sentimenDF = pd.read_csv(f"data/{ticker}/sentimen_preprocessing_{ticker}.csv")
sentimenDF

,tanggal,sentimen,clean_text,text_final
0,2022-07-01,@cahbola69 Sementara yg sudah fix brp sponsor ...,sementara yg sudah fix brp sponsor mas? kayan...,sementara sudah fix sponsor mas kayane lagi tok
1,2022-07-01,@360drajat @worksfess Sales apa dulu cobain de...,sales apa dulu cobain deh sales di perusahaa...,sales apa dulu coba deh sales usaha fmcg kayak...
2,2022-07-01,@indofood_cbp @Indomielovers Halo min @indofoo...,halo min saya ingin bertanya apakah saat in...,halo min ingin tanya apakah pt sedang buka aju...
3,2022-07-01,@lotekdevotee Indofood bakal bangkrut kayaknya...,indofood bakal bangkrut kayaknya dan bakal be...,bakal bangkrut kayak bakal dampak negara juga ...
4,2022-07-01,@MachanKertas Nu Indofood gak sih kalo lo.,nu indofood gak sih kalo lo,nu enggak sih kalo lo
...,...,...,...,...
274658,2025-10-11,😂,😂,NaN
274659,2025-10-11,Ke 7😊😊😊,ke <num> 😊😊😊,ke num
274660,2025-10-11,15 mnt yang lalu🎉,<num> mnt yang lalu🎉,num minta lalu
274661,2025-10-11,Keempat,keempat,empat


In [ ]:
sentimenDF = sentimenDF[["tanggal", "text_final"]].copy()
sentimenDF

,tanggal,text_final
0,2022-07-01,sementara sudah fix sponsor mas kayane lagi tok
1,2022-07-01,sales apa dulu coba deh sales usaha fmcg kayak...
2,2022-07-01,halo min ingin tanya apakah pt sedang buka aju...
3,2022-07-01,bakal bangkrut kayak bakal dampak negara juga ...
4,2022-07-01,nu enggak sih kalo lo
...,...,...
274658,2025-10-11,NaN
274659,2025-10-11,ke num
274660,2025-10-11,num minta lalu
274661,2025-10-11,empat


In [ ]:
sentimenDF.isna().sum()

,0
tanggal,0
text_final,15903


In [ ]:
sentimenDF.dropna(inplace=True)
sentimenDF = sentimenDF[sentimenDF['text_final'] != ''].reset_index(drop=True)

In [ ]:
sentimenDF

,tanggal,text_final
0,2022-07-01,sementara sudah fix sponsor mas kayane lagi tok
1,2022-07-01,sales apa dulu coba deh sales usaha fmcg kayak...
2,2022-07-01,halo min ingin tanya apakah pt sedang buka aju...
3,2022-07-01,bakal bangkrut kayak bakal dampak negara juga ...
4,2022-07-01,nu enggak sih kalo lo
...,...,...
258755,2025-10-11,ke num
258756,2025-10-11,ke num
258757,2025-10-11,num minta lalu
258758,2025-10-11,empat


In [ ]:
sentimenDF["prediksi_sentimen"] = sp.predict_sentimen(sentimenDF, sentimen_model, vectorizer, ticker)

100%|██████████| 52/52 [01:01<00:00,  1.18s/it]


In [ ]:
sentimenDF

,tanggal,text_final,prediksi_sentimen
0,2022-07-01,sementara sudah fix sponsor mas kayane lagi tok,1
1,2022-07-01,sales apa dulu coba deh sales usaha fmcg kayak...,1
2,2022-07-01,halo min ingin tanya apakah pt sedang buka aju...,1
3,2022-07-01,bakal bangkrut kayak bakal dampak negara juga ...,1
4,2022-07-01,nu enggak sih kalo lo,0
...,...,...,...
258755,2025-10-11,ke num,1
258756,2025-10-11,ke num,1
258757,2025-10-11,num minta lalu,1
258758,2025-10-11,empat,1


In [ ]:
sentimenDF.to_csv(f"data/{ticker}/sentimen_predict_{ticker}.csv", index=False)

### Integrasi Data

In [ ]:
print(ticker)
sentimenDF = pd.read_csv(f"data/{ticker}/sentimen_predict_{ticker}.csv")
sentimenDF

INDF


,tanggal,text_final,prediksi_sentimen
0,2022-07-01,sementara sudah fix sponsor mas kayane lagi tok,1
1,2022-07-01,sales apa dulu coba deh sales usaha fmcg kayak...,1
2,2022-07-01,halo min ingin tanya apakah pt sedang buka aju...,1
3,2022-07-01,bakal bangkrut kayak bakal dampak negara juga ...,1
4,2022-07-01,nu enggak sih kalo lo,0
...,...,...,...
258755,2025-10-11,ke num,1
258756,2025-10-11,ke num,1
258757,2025-10-11,num minta lalu,1
258758,2025-10-11,empat,1


In [ ]:
sentimenMAP = sentimenDF.copy()
mapping = {2: 1, 1 : 0, 0 : -1}

sentimenMAP['prediksi_sentimen'] = sentimenMAP['prediksi_sentimen'].map(mapping)
sentimenMAP

,tanggal,text_final,prediksi_sentimen
0,2022-07-01,sementara sudah fix sponsor mas kayane lagi tok,0
1,2022-07-01,sales apa dulu coba deh sales usaha fmcg kayak...,0
2,2022-07-01,halo min ingin tanya apakah pt sedang buka aju...,0
3,2022-07-01,bakal bangkrut kayak bakal dampak negara juga ...,0
4,2022-07-01,nu enggak sih kalo lo,-1
...,...,...,...
258755,2025-10-11,ke num,0
258756,2025-10-11,ke num,0
258757,2025-10-11,num minta lalu,0
258758,2025-10-11,empat,0


In [ ]:
sentimenDF['prediksi_sentimen'].value_counts()

,count
prediksi_sentimen,
1,115430
2,75009
0,68321


In [ ]:
sentimenMAP['prediksi_sentimen'].value_counts()

,count
prediksi_sentimen,
0,115430
1,75009
-1,68321


In [ ]:
sentimenMAP["tanggal"] = pd.to_datetime(sentimenMAP["tanggal"])

df_mean = (
    sentimenMAP
    .groupby("tanggal", as_index=False)["prediksi_sentimen"]
    .mean()
)

df_mean = df_mean.sort_values("tanggal")

df_mean.rename(columns={"prediksi_sentimen": "nilai_sentimen"}, inplace=True)

df_mean

,tanggal,nilai_sentimen
0,2013-01-19,0.500000
1,2014-12-07,0.000000
2,2015-09-12,0.500000
3,2016-05-27,0.375000
4,2016-05-28,0.111111
...,...,...
3277,2025-10-15,-0.117647
3278,2025-10-16,-0.022727
3279,2025-10-17,-0.300000
3280,2025-10-18,0.000000


In [ ]:
historisDF = pd.read_csv(
    f"./data/{ticker}/{ticker}.csv",
    index_col=0,
    parse_dates=[0],
    date_format="%Y-%m-%d"
    )

historisDF = historisDF[historisDF.index.str.match(r'\d{4}-\d{2}-\d{2}', na=False)]
historisDF.index = pd.to_datetime(historisDF.index)
historisDF.index.name = 'tanggal'

historisDF

,Close,High,Low,Open,Volume
tanggal,,,,,
2022-07-11,6016.36474609375,6016.36474609375,5930.72254330238,5994.954195395907,2961000
2022-07-12,6037.7744140625,6080.595509197695,5973.542771359707,6037.7744140625,5209400
2022-07-13,5845.080078125,5994.953926282052,5823.669528388278,5994.953926282052,10978800
2022-07-14,5930.72216796875,5952.132717311598,5823.669421254513,5845.07997059736,7611600
2022-07-15,5909.3125,5952.133605072464,5845.080842391304,5930.723052536232,4235000
...,...,...,...,...,...
2025-07-25,8400.0,8400.0,8300.0,8350.0,3388300
2025-07-28,8375.0,8475.0,8350.0,8475.0,3305800
2025-07-29,8300.0,8400.0,8275.0,8350.0,3728500


In [ ]:
integrasiDF = pd.merge(
    historisDF,
    df_mean,
    on="tanggal",
    how="left"
)

integrasiDF

,tanggal,Close,High,Low,Open,Volume,nilai_sentimen
0,2022-07-11,6016.36474609375,6016.36474609375,5930.72254330238,5994.954195395907,2961000,0.131148
1,2022-07-12,6037.7744140625,6080.595509197695,5973.542771359707,6037.7744140625,5209400,0.205047
2,2022-07-13,5845.080078125,5994.953926282052,5823.669528388278,5994.953926282052,10978800,0.204082
3,2022-07-14,5930.72216796875,5952.132717311598,5823.669421254513,5845.07997059736,7611600,0.259887
4,2022-07-15,5909.3125,5952.133605072464,5845.080842391304,5930.723052536232,4235000,0.334204
...,...,...,...,...,...,...,...
727,2025-07-25,8400.0,8400.0,8300.0,8350.0,3388300,-0.212928
728,2025-07-28,8375.0,8475.0,8350.0,8475.0,3305800,-0.064327
729,2025-07-29,8300.0,8400.0,8275.0,8350.0,3728500,-0.176030
730,2025-07-30,8375.0,8400.0,8275.0,8325.0,6796700,-0.089806


In [ ]:
integrasiDF.isna().sum()

,0
tanggal,0
Close,0
High,0
Low,0
Open,0
Volume,0
nilai_sentimen,0


In [ ]:
integrasiDF["nilai_sentimen"] = integrasiDF["nilai_sentimen"].fillna(0)

In [ ]:
integrasiDF = integrasiDF.set_index("tanggal")
integrasiDF

,Close,High,Low,Open,Volume,nilai_sentimen
tanggal,,,,,,
2022-07-11,6016.36474609375,6016.36474609375,5930.72254330238,5994.954195395907,2961000,0.131148
2022-07-12,6037.7744140625,6080.595509197695,5973.542771359707,6037.7744140625,5209400,0.205047
2022-07-13,5845.080078125,5994.953926282052,5823.669528388278,5994.953926282052,10978800,0.204082
2022-07-14,5930.72216796875,5952.132717311598,5823.669421254513,5845.07997059736,7611600,0.259887
2022-07-15,5909.3125,5952.133605072464,5845.080842391304,5930.723052536232,4235000,0.334204
...,...,...,...,...,...,...
2025-07-25,8400.0,8400.0,8300.0,8350.0,3388300,-0.212928
2025-07-28,8375.0,8475.0,8350.0,8475.0,3305800,-0.064327
2025-07-29,8300.0,8400.0,8275.0,8350.0,3728500,-0.176030


In [ ]:
integrasiDF.to_csv(f"data/{ticker}/data_integrasi_{ticker}.csv")

## GOTO

In [ ]:
ticker = 'GOTO'

### Load Data

In [ ]:
sentimenX = pd.read_csv(f"Scrapping_Data/{ticker}/data/Full_X_comments_{ticker}.csv")
sentimenX

,tanggal,sentimen
0,2022-07-01,Coba cek ini deh: Green leaf DAMICA Water Jug ...
1,2022-07-01,Coba cek ini deh: Travelling Towel Haiking Mer...
2,2022-07-01,Coba cek ini deh: EIGER PACK TOWEL A - Hijau M...
3,2022-07-01,Coba cek ini deh: Bantal Terapi Kompres Air Pa...
4,2022-07-01,Coba cek ini deh: BIYU Skin - Cleanser Bundle ...
...,...,...
79169,2025-07-31,Bawang Hitam Tunggal Premium Quality Alamme 10...
79170,2025-07-31,Hukum dan sistem kenegaraan sudah dirusak 10 t...
79171,2025-07-31,Coba lihat produk ini! https://t.co/Fo5wmaIxmW
79172,2025-07-31,Coba lihat produk ini! https://t.co/E2eMSeRA2x


In [ ]:
sentimenYT = pd.read_csv(f"Scrapping_Data/{ticker}/data/youtube_comments_{ticker}.csv")
sentimenYT

,tanggal,sentimen
0,2025-07-18,saham dushare disini auto turun gaes😂
1,2025-07-18,Kalo beli 4 4 nya judulnya boncos 4x 😂😂
2,2025-07-18,"Owalah Goto dishare disini , pantesan jadi tur..."
3,2025-07-18,next time please invite someone whose articula...
4,2025-07-18,❤❤❤😂😂😂😂😂
...,...,...
349734,2020-05-13,E3 nyimak bos. \nPembayaran pas jak kendaraan....
349735,2020-05-13,terima kasih kawan
349736,2020-05-13,Pasti bermanfaat seniorkuuu 😁✊
349737,2020-05-13,siap terimah kasih bro


In [ ]:
sentimenDF = pd.concat([sentimenX, sentimenYT], ignore_index=True)
sentimenDF

,tanggal,sentimen
0,2022-07-01,Coba cek ini deh: Green leaf DAMICA Water Jug ...
1,2022-07-01,Coba cek ini deh: Travelling Towel Haiking Mer...
2,2022-07-01,Coba cek ini deh: EIGER PACK TOWEL A - Hijau M...
3,2022-07-01,Coba cek ini deh: Bantal Terapi Kompres Air Pa...
4,2022-07-01,Coba cek ini deh: BIYU Skin - Cleanser Bundle ...
...,...,...
428908,2020-05-13,E3 nyimak bos. \nPembayaran pas jak kendaraan....
428909,2020-05-13,terima kasih kawan
428910,2020-05-13,Pasti bermanfaat seniorkuuu 😁✊
428911,2020-05-13,siap terimah kasih bro


### Cek data null dan duplikat

In [ ]:
sentimenDF.duplicated().sum()

np.int64(27638)

In [ ]:
sentimenDF.isna().sum()

,0
tanggal,0
sentimen,13


In [ ]:
sentimenDF.dropna(inplace=True)
sentimenDF.drop_duplicates(inplace=True)

In [ ]:
print(sentimenDF.duplicated().sum())
sentimenDF.isna().sum()

0


,0
tanggal,0
sentimen,0


### preprocessing data

In [ ]:
sentimenDF['clean_text'] = sp.preprocess_df(sentimenDF['sentimen'])

In [ ]:
sentimenDF

,tanggal,sentimen,clean_text
0,2022-07-01,Coba cek ini deh: Green leaf DAMICA Water Jug ...,coba cek ini deh green leaf damica water jug ...
1,2022-07-01,Coba cek ini deh: Travelling Towel Haiking Mer...,coba cek ini deh travelling towel haiking mer...
2,2022-07-01,Coba cek ini deh: EIGER PACK TOWEL A - Hijau M...,coba cek ini deh eiger pack towel a - hijau m...
3,2022-07-01,Coba cek ini deh: Bantal Terapi Kompres Air Pa...,coba cek ini deh bantal terapi kompres air pa...
4,2022-07-01,Coba cek ini deh: BIYU Skin - Cleanser Bundle ...,coba cek ini deh biyu skin - cleanser bundle ...
...,...,...,...
428908,2020-05-13,E3 nyimak bos. \nPembayaran pas jak kendaraan....,e<num> nyimak bos \npembayaran pas jak kendar...
428909,2020-05-13,terima kasih kawan,terima kasih kawan
428910,2020-05-13,Pasti bermanfaat seniorkuuu 😁✊,pasti bermanfaat seniorkuuu 😁✊
428911,2020-05-13,siap terimah kasih bro,siap terimah kasih bro


In [ ]:
sentimenDF = sp.parallel_processing(sentimenDF)

Memproses 401267 data dengan 2 pekerja.


🔄 Main Progress: 100%|██████████| 21/21 [7:01:37<00:00, 1204.63s/it]


In [ ]:
sentimenDF

,tanggal,sentimen,clean_text,text_final
0,2022-07-01,@cahbola69 Sementara yg sudah fix brp sponsor ...,sementara yg sudah fix brp sponsor mas? kayan...,sementara sudah fix sponsor mas kayane lagi tok
1,2022-07-01,@360drajat @worksfess Sales apa dulu cobain de...,sales apa dulu cobain deh sales di perusahaa...,sales apa dulu coba deh sales usaha fmcg kayak...
2,2022-07-01,@indofood_cbp @Indomielovers Halo min @indofoo...,halo min saya ingin bertanya apakah saat in...,halo min ingin tanya apakah pt sedang buka aju...
3,2022-07-01,@lotekdevotee Indofood bakal bangkrut kayaknya...,indofood bakal bangkrut kayaknya dan bakal be...,bakal bangkrut kayak bakal dampak negara juga ...
4,2022-07-01,@MachanKertas Nu Indofood gak sih kalo lo.,nu indofood gak sih kalo lo,nu enggak sih kalo lo
...,...,...,...,...
289422,2025-10-11,😂,😂,
289423,2025-10-11,Ke 7😊😊😊,ke <num> 😊😊😊,ke num
289424,2025-10-11,15 mnt yang lalu🎉,<num> mnt yang lalu🎉,num minta lalu
289425,2025-10-11,Keempat,keempat,empat


### Simpan Data

In [ ]:
sentimenDF.to_csv(f"data/{ticker}/sentimen_preprocessing_{ticker}.csv", index=False)

### prediksi nilai sentimen

In [ ]:
sentimenDF = pd.read_csv(f"data/{ticker}/sentimen_preprocessing_{ticker}.csv")
sentimenDF

,tanggal,sentimen,clean_text,text_final
0,2022-07-01,Coba cek ini deh: Green leaf DAMICA Water Jug ...,coba cek ini deh green leaf damica water jug ...,coba cek deh green leaf damica water jug teko ...
1,2022-07-01,Coba cek ini deh: Travelling Towel Haiking Mer...,coba cek ini deh travelling towel haiking mer...,coba cek deh travelling towel haiking rapi mou...
2,2022-07-01,Coba cek ini deh: EIGER PACK TOWEL A - Hijau M...,coba cek ini deh eiger pack towel a - hijau m...,coba cek deh eiger pack towel a hijau sama rp ...
3,2022-07-01,Coba cek ini deh: Bantal Terapi Kompres Air Pa...,coba cek ini deh bantal terapi kompres air pa...,coba cek deh bantal terapi kompres air panas e...
4,2022-07-01,Coba cek ini deh: BIYU Skin - Cleanser Bundle ...,coba cek ini deh biyu skin - cleanser bundle ...,coba cek deh biyu skin cleanser bundle num pcs...
...,...,...,...,...
401262,2020-05-13,E3 nyimak bos. \nPembayaran pas jak kendaraan....,e<num> nyimak bos \npembayaran pas jak kendar...,e num simak bos bayar pas jak kendara moga suk...
401263,2020-05-13,terima kasih kawan,terima kasih kawan,terima kasih kawan
401264,2020-05-13,Pasti bermanfaat seniorkuuu 😁✊,pasti bermanfaat seniorkuuu 😁✊,pasti manfaat seniorkuuu
401265,2020-05-13,siap terimah kasih bro,siap terimah kasih bro,terimah kasih bro


In [ ]:
sentimenDF = sentimenDF[["tanggal", "text_final"]].copy()
sentimenDF

,tanggal,text_final
0,2022-07-01,coba cek deh green leaf damica water jug teko ...
1,2022-07-01,coba cek deh travelling towel haiking rapi mou...
2,2022-07-01,coba cek deh eiger pack towel a hijau sama rp ...
3,2022-07-01,coba cek deh bantal terapi kompres air panas e...
4,2022-07-01,coba cek deh biyu skin cleanser bundle num pcs...
...,...,...
401262,2020-05-13,e num simak bos bayar pas jak kendara moga suk...
401263,2020-05-13,terima kasih kawan
401264,2020-05-13,pasti manfaat seniorkuuu
401265,2020-05-13,terimah kasih bro


In [ ]:
sentimenDF.isna().sum()

,0
tanggal,0
text_final,8936


In [ ]:
sentimenDF.dropna(inplace=True)
sentimenDF = sentimenDF[sentimenDF['text_final'] != ''].reset_index(drop=True)

In [ ]:
sentimenDF

,tanggal,text_final
0,2022-07-01,coba cek deh green leaf damica water jug teko ...
1,2022-07-01,coba cek deh travelling towel haiking rapi mou...
2,2022-07-01,coba cek deh eiger pack towel a hijau sama rp ...
3,2022-07-01,coba cek deh bantal terapi kompres air panas e...
4,2022-07-01,coba cek deh biyu skin cleanser bundle num pcs...
...,...,...
392326,2020-05-13,e num simak bos bayar pas jak kendara moga suk...
392327,2020-05-13,terima kasih kawan
392328,2020-05-13,pasti manfaat seniorkuuu
392329,2020-05-13,terimah kasih bro


In [ ]:
sentimenDF["prediksi_sentimen"] = sp.predict_sentimen(sentimenDF, sentimen_model, vectorizer, ticker)

100%|██████████| 79/79 [01:43<00:00,  1.31s/it]


In [ ]:
sentimenDF

,tanggal,text_final,prediksi_sentimen
0,2022-07-01,coba cek deh green leaf damica water jug teko ...,2
1,2022-07-01,coba cek deh travelling towel haiking rapi mou...,2
2,2022-07-01,coba cek deh eiger pack towel a hijau sama rp ...,2
3,2022-07-01,coba cek deh bantal terapi kompres air panas e...,1
4,2022-07-01,coba cek deh biyu skin cleanser bundle num pcs...,2
...,...,...,...
392326,2020-05-13,e num simak bos bayar pas jak kendara moga suk...,2
392327,2020-05-13,terima kasih kawan,1
392328,2020-05-13,pasti manfaat seniorkuuu,2
392329,2020-05-13,terimah kasih bro,2


In [ ]:
sentimenDF.to_csv(f"data/{ticker}/sentimen_predict_{ticker}.csv", index=False)

### Integrasi Data

In [ ]:
print(ticker)
sentimenDF = pd.read_csv(f"data/{ticker}/sentimen_predict_{ticker}.csv")
sentimenDF

GOTO


,tanggal,text_final,prediksi_sentimen
0,2022-07-01,coba cek deh green leaf damica water jug teko ...,2
1,2022-07-01,coba cek deh travelling towel haiking rapi mou...,2
2,2022-07-01,coba cek deh eiger pack towel a hijau sama rp ...,2
3,2022-07-01,coba cek deh bantal terapi kompres air panas e...,1
4,2022-07-01,coba cek deh biyu skin cleanser bundle num pcs...,2
...,...,...,...
392326,2020-05-13,e num simak bos bayar pas jak kendara moga suk...,2
392327,2020-05-13,terima kasih kawan,1
392328,2020-05-13,pasti manfaat seniorkuuu,2
392329,2020-05-13,terimah kasih bro,2


In [ ]:
sentimenMAP = sentimenDF.copy()
mapping = {2: 1, 1 : 0, 0 : -1}

sentimenMAP['prediksi_sentimen'] = sentimenMAP['prediksi_sentimen'].map(mapping)
sentimenMAP

,tanggal,text_final,prediksi_sentimen
0,2022-07-01,coba cek deh green leaf damica water jug teko ...,1
1,2022-07-01,coba cek deh travelling towel haiking rapi mou...,1
2,2022-07-01,coba cek deh eiger pack towel a hijau sama rp ...,1
3,2022-07-01,coba cek deh bantal terapi kompres air panas e...,0
4,2022-07-01,coba cek deh biyu skin cleanser bundle num pcs...,1
...,...,...,...
392326,2020-05-13,e num simak bos bayar pas jak kendara moga suk...,1
392327,2020-05-13,terima kasih kawan,0
392328,2020-05-13,pasti manfaat seniorkuuu,1
392329,2020-05-13,terimah kasih bro,1


In [ ]:
sentimenDF['prediksi_sentimen'].value_counts()

,count
prediksi_sentimen,
1,175409
0,124833
2,92089


In [ ]:
sentimenMAP['prediksi_sentimen'].value_counts()

,count
prediksi_sentimen,
0,175409
-1,124833
1,92089


In [ ]:
sentimenMAP["tanggal"] = pd.to_datetime(sentimenMAP["tanggal"])

df_mean = (
    sentimenMAP
    .groupby("tanggal", as_index=False)["prediksi_sentimen"]
    .mean()
)

df_mean = df_mean.sort_values("tanggal")

df_mean.rename(columns={"prediksi_sentimen": "nilai_sentimen"}, inplace=True)

df_mean

,tanggal,nilai_sentimen
0,2015-10-21,0.066667
1,2015-10-22,0.000000
2,2015-10-23,1.000000
3,2015-10-24,-0.800000
4,2015-10-26,-0.500000
...,...,...
2674,2025-10-15,-0.200000
2675,2025-10-16,0.250000
2676,2025-10-17,-0.222222
2677,2025-10-18,-0.181818


In [ ]:
historisDF = pd.read_csv(
    f"./data/{ticker}/{ticker}.csv",
    index_col=0,
    parse_dates=[0],
    date_format="%Y-%m-%d"
    )

historisDF = historisDF[historisDF.index.str.match(r'\d{4}-\d{2}-\d{2}', na=False)]
historisDF.index = pd.to_datetime(historisDF.index)
historisDF.index.name = 'tanggal'

historisDF

,Close,High,Low,Open,Volume
tanggal,,,,,
2022-07-11,338.0,354.0,334.0,354.0,840928300
2022-07-12,330.0,338.0,330.0,338.0,802215500
2022-07-13,324.0,330.0,320.0,330.0,870421000
2022-07-14,332.0,336.0,320.0,324.0,786358600
2022-07-15,320.0,336.0,320.0,332.0,998698500
...,...,...,...,...,...
2025-07-25,58.0,59.0,57.0,59.0,5090425000
2025-07-28,59.0,60.0,58.0,58.0,3567335000
2025-07-29,59.0,60.0,58.0,59.0,1423009300


In [ ]:
integrasiDF = pd.merge(
    historisDF,
    df_mean,
    on="tanggal",
    how="left"
)

integrasiDF

,tanggal,Close,High,Low,Open,Volume,nilai_sentimen
0,2022-07-11,338.0,354.0,334.0,354.0,840928300,0.086957
1,2022-07-12,330.0,338.0,330.0,338.0,802215500,0.127820
2,2022-07-13,324.0,330.0,320.0,330.0,870421000,-0.022727
3,2022-07-14,332.0,336.0,320.0,324.0,786358600,-0.090032
4,2022-07-15,320.0,336.0,320.0,332.0,998698500,-0.024390
...,...,...,...,...,...,...,...
727,2025-07-25,58.0,59.0,57.0,59.0,5090425000,-0.165906
728,2025-07-28,59.0,60.0,58.0,58.0,3567335000,-0.139048
729,2025-07-29,59.0,60.0,58.0,59.0,1423009300,-0.040984
730,2025-07-30,64.0,64.0,58.0,59.0,6540042400,-0.031359


In [ ]:
integrasiDF.isna().sum()

,0
tanggal,0
Close,0
High,0
Low,0
Open,0
Volume,0
nilai_sentimen,0


In [ ]:
integrasiDF["nilai_sentimen"] = integrasiDF["nilai_sentimen"].fillna(0)

In [ ]:
integrasiDF = integrasiDF.set_index("tanggal")
integrasiDF

,Close,High,Low,Open,Volume,nilai_sentimen
tanggal,,,,,,
2022-07-11,338.0,354.0,334.0,354.0,840928300,0.086957
2022-07-12,330.0,338.0,330.0,338.0,802215500,0.127820
2022-07-13,324.0,330.0,320.0,330.0,870421000,-0.022727
2022-07-14,332.0,336.0,320.0,324.0,786358600,-0.090032
2022-07-15,320.0,336.0,320.0,332.0,998698500,-0.024390
...,...,...,...,...,...,...
2025-07-25,58.0,59.0,57.0,59.0,5090425000,-0.165906
2025-07-28,59.0,60.0,58.0,58.0,3567335000,-0.139048
2025-07-29,59.0,60.0,58.0,59.0,1423009300,-0.040984


In [ ]:
integrasiDF.to_csv(f"data/{ticker}/data_integrasi_{ticker}.csv")